# Closed loop laser scan with NI-DAQmx

## Setup

In [1]:
import time
import numpy as np
import socket

from pylabnet.utils.logging.logger import LogClient
from pylabnet.scripts.lasers import wlm_monitor
from pylabnet.network.client_server import sweeper, si_tt_cnt_monitor
from pylabnet.utils.sweeper import Sweep1D, MultiChSweep1D
from pylabnet.utils.helper_methods import create_server

Enter host and port parameters here:

In [2]:
logger_host = '140.247.189.94'
logger_port = 36072
wavemeter_host = '140.247.189.94'
wavemeter_port = 11172
ctr_host = '140.247.189.82'
ctr_port = 16555

Instantiate all clients

In [3]:
exp_log = LogClient(
    host=logger_host,
    port=logger_port,
    module_tag='laser_scan_closed_loop'
)
wm = wlm_monitor.Client(
    host=wavemeter_host,
    port=wavemeter_port,
)
ctr = si_tt_cnt_monitor.Client(
    host=ctr_host,
    port=ctr_port
)
ctr.set_channels(ch_list=[1])
ctr.start_counting(n_bins=1)
exp = Sweep1D(logger=exp_log)

In [4]:
update_log = LogClient(
    host=logger_host,
    port=logger_port,
    module_tag='1D_update_server'
)
pause_1d_service = sweeper.Service()
pause_1d_service.assign_module(module=exp)
pause_1d_service.assign_logger(logger=update_log)
pause_1d_server, pause_1d_port = create_server(
    service=pause_1d_service,
    logger=update_log,
    host=socket.gethostbyname(socket.gethostname())
)
update_log.update_data(data={'port': pause_1d_port})
pause_1d_server.start()

## Experiment

Parameters, setup

In [5]:
f0 = 406.700
f1 = 406.702
pts = 101
exp.set_parameters(min=f0, max=f1, pts=pts)

Experiment script

In [9]:
integrate = 0.1
def experiment(frequency):

    # Set frequency
    wm.update_parameters([{
        'channel': 6, 'setpoint': frequency
    }])

    #Activate counter
    ctr.clear_counter()
    time.sleep(integrate)
    counts = ctr.get_counts()

    return np.sum(counts)/integrate
    

In [10]:
exp.configure_experiment(experiment)

In [11]:
exp.set_reps(reps=10)
exp.run(plot=True, autosave=False)

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Single',
              'type': 'scatter',…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '0603d958-a034-4930-adfc-7d1d65a810fe',
 …

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Single',
              'type': 'scatter',…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '1fff1dad-f7ce-46a0-9ce1-228da1b3131f',
 …

## Multi-channel sweep

In [17]:
mc_log = LogClient(
    host=logger_host,
    port=logger_port,
    module_tag='laser_scan_closed_loop_multichannel'
)

mc_pause_log = LogClient(
    host=logger_host,
    port=logger_port,
    module_tag='MultiCh1D_pause_server'
)

m_exp = MultiChSweep1D(logger=mc_log, channels=['Transmission', 'Reflection'])
pause_m_service = sweeper.Service()
pause_m_service.assign_module(module=exp)
pause_m_service.assign_logger(logger=mc_pause_log)
pause_m_server = create_server(
    service=pause_m_service,
    logger=mc_pause_log,
    host=logger_host
)

In [18]:
m_exp.set_parameters(min=f0, max=f1, pts=pts, reps=4)

In [19]:
def multi_experiment(frequency):
    time.sleep(0.02)
    diffusion = 0.0002*np.sin(2*np.pi*time.time()/50)
    return [
        100 + 0.01/((frequency-406.694+diffusion)**2+(0.0001)**2),
        100 - 0.01/((frequency-406.694+diffusion)**2+(0.0001)**2)
    ]
m_exp.configure_experiment(multi_experiment)

In [20]:
m_exp.run(plot=True)

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Transmission Single',
              'type…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': 'a36df67b-3158-49db-a510-0af4474ca83f',
 …

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Transmission Single',
              'type…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '9edc977a-61b7-4b8e-ba4b-46b5c4b7e1f6',
 …

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Reflection Single',
              'type':…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '1825d0c6-bb91-4049-97b0-de0e9d2c5364',
 …

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Reflection Single',
              'type':…

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': 'b8187589-e1e3-46f5-ba41-4f153018ad26',
 …

## Close counter

In [5]:
#ni.close_timed_counter('counter_0')